In [ ]:
import os
from simple_cysh import * # includes pandas as pd

In [ ]:
program_df = get_cysh_df('Program__c', ['Id', 'Name'], 
                         rename_id=True)
program_df.rename(columns={'Name': 'Program'},inplace=True)

# get sections, merge in program name, filter for sections of interest
section_df = get_cysh_df('Section__c', ['Id', 'School__c', 'Program__c', 'In_After_School__c'], 
                         rename_id=True, 
#                          where="Active__c = True"
                        )
section_df = section_df.merge(program_df, on='Program__c', how='left')

section_df = section_df.loc[section_df['Program'].isin([
#     'Coaching: Attendance', 
#     'Tutoring: Literacy', 
#     'Tutoring: Math',
#     'SEL Check In Check Out',
    'Enrichment',
])]

stu_sec_df = get_cysh_df('Student_Section__c', ['Id', 'Section__c', 'Student__c', 'Student_Id__c', 
                                                'Student_Name__c', 'Student_Grade__c', 
                                                'Amount_of_Time__c', 'Intervention_Enrollment_Start_Date__c', 
                                                'Enrollment_End_Date__c', 'Section_Exit_Reason__c'],
                         rename_id=True, 
#                          where="Active__c = True"
                        )
stu_sec_df = stu_sec_df.loc[~stu_sec_df['Student_Name__c'].str.contains('DELETE')]
stu_lastnames = get_cysh_df('Student__c', ['Id', 'Student_Last_Name__c'], rename_id=True)
stu_sec_df = stu_sec_df.merge(stu_lastnames, how='left', on='Student__c')

school_df = get_cysh_df('Account', ['Id', 'Name'])
school_df.rename(columns={'Id':'School__c', 'Name':'School'}, inplace=True)

In [ ]:
# merges
df = section_df.merge(school_df, how='left', on='School__c')
df = df.merge(stu_sec_df, on='Section__c')

asp_counts = df.groupby('School')['Student__c'].nunique().reset_index()

In [ ]:
asp_counts.sort_values('Student__c')